# EVA tool

## __E__valuation and __V__alidation __A__nalysis tool

### Main steps:

- read CSV file

#### 1 - Plotting and analyse opportunity traffic
- sort plots by track_ID
- for each track look for the gaps and create new plots to fill them in (missed plots)
- calculate statistics: % of missed and % of probability of detection
- plot every track marking the real plots and the missed ones.
- implement a picker to display plot and track information when pick over a plot

#### 2 - Accuracy analysis by using a GPS points file
- read the GPS track file
- select/introduce the track/tracks to compare with the GPS points
- correlate every plot with the time nearest GPS point
- calculate the positional error for every plot
- statistical analisys of the error
- plot an X/Y absolute error

import modules:

In [1]:
%matplotlib qt 
#inline
%config InlineBackend.figure_format = 'retina'

import matplotlib
matplotlib.use('TkAgg')  

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/angelvidal/anaconda3/envs/CarND-TensorFlow-L2/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [10]:
ll *.csv


-rw-r--r--  1 angelvidal  staff  7878870 Feb 24 21:05 080001.gps.csv
-rw-r--r--  1 angelvidal  staff  7878871 Feb 24 19:50 080001.gps.sorted.csv
-rw-r--r--  2 angelvidal  staff    32881 Dec 29  2015 statistics.csv


In [11]:
# this should be implemented by means of a dialog 
file = input("Enter the filename: ")

Enter the filename: ñasldkfj


In [2]:
# read de csv file
data = pd.read_csv('080001.gps.csv')
data.head(3)

,Cat,Length,FSPEC,DI_I010_010_SAC,DI_I010_010_SIC,DI_I010_000_Message_Type,DI_I010_020_TRD_Type,DI_I010_020_TRD_DCR,DI_I010_020_TRD_CHN,DI_I010_020_TRD_GBS,...,DI_I010_550_System_Status_TTF,DI_I010_310_PreProgrammed_Message,DI_I010_550_Standard_Deviation_of_Position_sigmaX,DI_I010_550_Standard_Deviation_of_Position_sigmaY,DI_I010_550_Standard_Deviation_of_Position_covariXY,DI_I010_280_Presence,DI_I010_131_Amplitude_of_Primary_Plot,DI_I010_210_Calculated_Acceleration,SP_Special_Purpose_Field,RE_Reserved_Expansion_Field
0,10,52,[247 241 9 48],1,0,Target Report,PSR,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,20,[208],1,0,Start of Update Cycle,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,52,[247 241 9 48],1,0,Target Report,PSR,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# get just the fields of interest into a new subset
subset = data.loc[:,['Cat', 'DI_I010_140_Time_of_Day',
                     'ADD_ON_time_at_recorder',
                     'ADD_ON_delta(t)',
                     'DI_I010_161_Track_No',
                     'DI_I010_042_X',
                     'DI_I010_042_Y',
                     'DI_I010_040_Rho',
                     'DI_I010_040_Theta',
                     'DI_I010_041_Lat',
                     'DI_I010_041_Lon',
                     'DI_I010_020_TRD_Type',
                     'DI_I010_000_Message_Type']]
# clean memory
del data

# show the head
subset.head()

,Cat,DI_I010_140_Time_of_Day,ADD_ON_time_at_recorder,ADD_ON_delta(t),DI_I010_161_Track_No,DI_I010_042_X,DI_I010_042_Y,DI_I010_040_Rho,DI_I010_040_Theta,DI_I010_041_Lat,DI_I010_041_Lon,DI_I010_020_TRD_Type,DI_I010_000_Message_Type
0,10,28801.554688,28801.648438,NaN,2766.0,-3806.0,1533.0,4104.0,-68.422852,NaN,NaN,PSR,Target Report
1,10,28801.835938,28801.835938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Start of Update Cycle
2,10,28802.546875,28802.640625,NaN,2766.0,-3874.0,1555.0,4175.0,-68.488770,NaN,NaN,PSR,Target Report
3,10,28802.835938,28802.828125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Start of Update Cycle
4,10,28803.554688,28803.648438,NaN,2766.0,-3942.0,1576.0,4246.0,-68.565674,NaN,NaN,PSR,Target Report


In [4]:
# delay analysis
delay = subset.ADD_ON_time_at_recorder - subset.DI_I010_140_Time_of_Day
print('delay minimum: ', np.min(delay))
print('delay maximum: ', np.max(delay))
print('delay mean/average: ', np.mean(delay))
print('delay histogram:')
plt.figure(figsize=(10,3))
plt.hist(delay, bins=100, color='orange')
plt.show()

delay minimum:  -0.0078125
delay maximum:  1.1015625
delay mean/average:  0.0562481391293
delay histogram:


In [5]:
# sort by track ID
subset = subset.sort_values(by='DI_I010_140_Time_of_Day')
subset.head()

,Cat,DI_I010_140_Time_of_Day,ADD_ON_time_at_recorder,ADD_ON_delta(t),DI_I010_161_Track_No,DI_I010_042_X,DI_I010_042_Y,DI_I010_040_Rho,DI_I010_040_Theta,DI_I010_041_Lat,DI_I010_041_Lon,DI_I010_020_TRD_Type,DI_I010_000_Message_Type
0,10,28801.554688,28801.648438,NaN,2766.0,-3806.0,1533.0,4104.0,-68.422852,NaN,NaN,PSR,Target Report
1,10,28801.835938,28801.835938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Start of Update Cycle
2,10,28802.546875,28802.640625,NaN,2766.0,-3874.0,1555.0,4175.0,-68.488770,NaN,NaN,PSR,Target Report
3,10,28802.835938,28802.828125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Start of Update Cycle
4,10,28803.554688,28803.648438,NaN,2766.0,-3942.0,1576.0,4246.0,-68.565674,NaN,NaN,PSR,Target Report


In [6]:
# counting plots in every track and stores in a dict {track_id: {counts : count, X: [], Y: [],... }}
tracks = {}
for plot, x, y, t in zip(subset.DI_I010_161_Track_No,
                         subset.DI_I010_042_X,
                         subset.DI_I010_042_Y,
                         subset.DI_I010_140_Time_of_Day,):
    if not np.isnan(plot):
        if plot in tracks:
            tracks[plot]['count'] += 1
            tracks[plot]['X'].append(x)
            tracks[plot]['Y'].append(y)
            tracks[plot]['T'].append(t)
            
        else:
            tracks[plot] = {}
            tracks[plot]['count'] = 1
            tracks[plot]['X'] = []
            tracks[plot]['Y'] = []
            tracks[plot]['T'] = []
            tracks[plot]['X'].append(x)
            tracks[plot]['Y'].append(y)            
            tracks[plot]['T'].append(t)            


In [7]:
plt.figure(figsize=(10, 5))
for track in tracks.keys():
    plt.plot(tracks[track]['X'], tracks[track]['Y'])
    

plt.grid(True)
plt.show()
